In [17]:
import tensorflow as tf

In [18]:
import pandas as pd
import requests

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/train.csv')

In [20]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [21]:
df= df.dropna()

In [22]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [23]:
x = df.drop('target',axis  = 1)

In [24]:
y = df.target

In [25]:
import tensorflow as tf

In [26]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers  import Dense

In [27]:
voc_size=5000

In [28]:
messages=x.copy()

In [29]:
messages.reset_index(inplace=True)

In [30]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [31]:


ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  text = re.sub('[^a-zA-Z]', ' ',messages.text[i]).lower().split()
  text = [ ps.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  corpus.append(text)

In [32]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [33]:
sent_length = 0
for x in onehot_rep:
  if sent_length<len(x):
    sent_length = len(x)


In [34]:
embed = pad_sequences(onehot_rep,padding = 'pre' ,maxlen = sent_length)

In [35]:
vector_features = 40
model=Sequential()
model.add(Embedding(voc_size,vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [36]:
import numpy as np
x_final = np.array(embed)
y_final = np.array(y)

In [37]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [38]:

model.fit(x_final,y_final,epochs=10,batch_size=64)

Epoch 1/10
80/80 [==============================] - 5s 34ms/step - loss: 0.6625 - accuracy: 0.6008
Epoch 2/10
80/80 [==============================] - 3s 33ms/step - loss: 0.3928 - accuracy: 0.8275
Epoch 3/10
80/80 [==============================] - 3s 33ms/step - loss: 0.2733 - accuracy: 0.8929
Epoch 4/10
80/80 [==============================] - 3s 33ms/step - loss: 0.1986 - accuracy: 0.9303
Epoch 5/10
80/80 [==============================] - 3s 33ms/step - loss: 0.1505 - accuracy: 0.9501
Epoch 6/10
80/80 [==============================] - 3s 33ms/step - loss: 0.1196 - accuracy: 0.9615
Epoch 7/10
80/80 [==============================] - 3s 34ms/step - loss: 0.0859 - accuracy: 0.9754
Epoch 8/10
80/80 [==============================] - 3s 34ms/step - loss: 0.0657 - accuracy: 0.9829
Epoch 9/10
80/80 [==============================] - 3s 33ms/step - loss: 0.0442 - accuracy: 0.9883
Epoch 10/10
80/80 [==============================] - 3s 33ms/step - loss: 0.0472 - accuracy: 0.9873


In [92]:
df = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/test.csv')


3263

In [93]:


ps = PorterStemmer()
corpus = []
for i in range(len(df)):
  text = re.sub('[^a-zA-Z]', ' ',df.text[i]).lower().split()
  text = [ ps.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  corpus.append(text)

sent_length = 0
for x in onehot_rep:
  if sent_length<len(x):
    sent_length = len(x)

In [94]:
embed = pad_sequences(onehot_rep,padding = 'pre' ,maxlen = sent_length)
x_test = np.array(embed)

res = model.predict(x_test)


In [95]:
res.mean()


0.42522728

In [96]:

pred = []

for id,val in zip(df.id,res):
  if val > 5:
    pred+=[[id,1]]
  else:
    pred+=[[id,0]]

In [97]:
import csv

with open('prediction.csv','w') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(['id','target'])
  for row in pred:
     csvwriter.writerow(row)

In [91]:
len(df)

2158